In [24]:
import csv

# Removes all none ASCII characters and replaces them with spaces
with open('data/pickHistoryOrderNo.csv', 'r', encoding='utf-8', errors='ignore') as file_in, open('data/output_file.csv', 'w', newline='', encoding='utf-8') as file_out:
    reader = csv.reader(file_in)
    writer = csv.writer(file_out)

    # Write the header row to the output file
    header = next(reader)
    writer.writerow(header)

    # Iterate over the rows in the input file and clean each row before writing it to the output file
    for row in reader:
        cleaned_row = []
        for cell in row:
            # Replace any non-unicode characters with spaces
            cleaned_cell = ''.join([char if ord(char) < 128 else ' ' for char in cell])
            cleaned_row.append(cleaned_cell)
        writer.writerow(cleaned_row)


In [25]:
# Remove whitespace and tabs

import csv

with open('data/output_file.csv', 'r', encoding='utf-8', errors='ignore') as file_in, open('data/leanPicks.csv', 'w', newline='', encoding='utf-8') as file_out:
    reader = csv.reader(file_in, delimiter='\t')
    writer = csv.writer(file_out)

    # Write the header row to the output file
    header = next(reader)
    header = [col.replace(' ', '').strip() for col in header]  # Remove whitespace and tab in column names
    writer.writerow(header)

    # Iterate over the rows in the input file, remove any whitespace or tabs, and write to the output file
    for row in reader:
        cleaned_row = [cell.replace(' ', '').strip() for cell in row]
        writer.writerow(cleaned_row)


In [26]:
# List refactored column headers

import pandas as pd

df = pd.read_csv("data/output_file.csv", on_bad_lines="skip")
df.columns.values.tolist()

['ISELL_ORDER_NUMBER\tPICK_ID\tARTNO   \tARTNAME_UNICODE                                   \tART_VOLUME_M3\tORDERED_QTY\tPICKED_QTY\tOPEN_PICK_QTY\tAVAILABLE_STOCK\tPOSSIBLE_TO_FINISH\tORDER_TYPE \tDATE_OF_PAYMENT\tTIME_OF_PAYMENT\tEXCEPTION\tPICK_AREA          \tACTUAL_ORDER_STATUS\tSTORAGE_STATUS\tSTORAGE_USED\tHANDOVER_POINT\tCUT_OFF_DATE\tCUT_OFF_TIME\tUSER_PICKING\tSERVICE_DATE\tSERVICE_WINDOW\tORDER_METHOD\tPICK_LOCATION\tPICK_LOCATION_TYPE\tDELIVERY_METHOD                  ']

In [27]:
# Create a new csv that contains only the columns we are interested in

# Read the CSV file into a DataFrame and specify a custom data type for the 'ARTNO' column
df = pd.read_csv('data/leanPicks.csv', usecols=['ISELL_ORDER_NUMBER', 'PICK_ID', 'ARTNAME_UNICODE', 'ORDER_TYPE', 'PICK_AREA', 'ARTNO'], dtype={'ARTNO': str})

# Write the selected columns to a new CSV file
df.to_csv('data/output_file.csv', index=False)


In [28]:
# Create a new csv that has all orders and the articles attributed to that order

import pandas as pd

# Read csv file
df = pd.read_csv('data/output_file.csv', dtype={'ARTNO': str})

# Convert ARTNO to string
df['ARTNO'] = df['ARTNO'].astype(str)

# Group by order number and concatenate article numbers
orders = df.groupby('ISELL_ORDER_NUMBER')['ARTNO'].apply(lambda x: ','.join(x)).reset_index()

# Rename columns
orders.columns = ['order_number', 'articles']

# Write to new csv file
orders.to_csv('data/orders.csv', index=False)



In [29]:
# Strip quotation marks
import csv


with open('data/orders.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    with open('data/strippedOrders.csv', 'w', newline='') as newfile:
        csvwriter = csv.writer(newfile)

        for row in csvreader:
            newrow = [cell.replace('"', '') for cell in row]
            csvwriter.writerow(newrow)


In [30]:
# Returns all articles that are included in an order. 

x = 1345368239

df = pd.read_csv('data/strippedOrders.csv', dtype={'ARTNO': str})

filtered_df = df[df['order_number'] == x]

articles = filtered_df.iloc[0]['articles'].split(",")

print(articles)

['20214566', '30256891', '20246708', '70277957', '70214564', '40256895', '70346932', '30246722']


In [31]:
# List of all unique articles

# Split the values in the 'articles' column based on the comma separator, and create a new row for each value
df['articles'] = df['articles'].str.split(',')
df = df.explode('articles')

# Get a list of unique values in the 'articles' column
unique_articles = df['articles'].unique().tolist()

# Write the list of unique articles to a new CSV file
pd.DataFrame({'articles': unique_articles}).to_csv('data/unique_articles.csv', index=False)

In [32]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('data/unique_articles.csv')

# Preprocess the 'ARTNO' column to add leading zeroes to the column values
df['articles'] = df['articles'].apply(lambda x: f'{x:0>8}')

# Iterate through each row of the DataFrame and each column of the row, and print the value of each cell
for index, row in df.iterrows():
    for column, value in row.items():
        orderdf = pd.read_csv('data/orders.csv')
        filtered_df = orderdf[orderdf['articles'].str.contains(value, na=False, case=False)]
        print("picks containing ", value, filtered_df['articles'].values)


picks containing  70518513 ['70518513']
picks containing  10242739 ['10242739']
picks containing  00527158 ['00527158']
picks containing  60532208 ['60532208']
picks containing  20399895 ['20399895']
picks containing  80251513 ['80251513']
picks containing  70532062 ['70532062'
 '00177225,00542998,20132453,20448834,20508408,20543001,30213552,40323654,40508407,40508884,50213546,50512649,70213545,70466811,70532062,90213549,90279719,90538001,20312492,40448036']
picks containing  60509142 ['60509142'
 '70509132,30214504,20510939,90509145,10509125,60509142,80256940,10263232,40257296'
 '80256940,60509142']
picks containing  60185034 ['60185034,60503569' '00458697,50458831,60185034,90472340']
picks containing  60503569 ['60185034,60503569']
picks containing  20214566 ['20214566,30256891,20246708,70277957,70214564,40256895,70346932,30246722'
 '10446883,10529086,60342656,60529084,70529088,10446557,00214505,10368489,20214566,20329195,30256891,30277959,70214559,70246386,70277957,70346932,80331930

In [33]:
# Find correlation between articles
import csv
import pandas as pd

# Must be stored as string
seen = ['20214566']

df = pd.read_csv('data/orders.csv', dtype={'articles': str})

# Gets all picks that contain a certain article
def findPicksWithArticle(df, x):
    # Filter the DataFrame to only include rows where the 'articles' column contains the specified value
    filtered_df = df[df['articles'].str.contains(x, na=False, case=False)]
    # Select the 'articles' column from the filtered DataFrame and return it
    artno_column = filtered_df[['articles']]

    relevant_picks = []

    for pick in artno_column.values:
        pickList = str(pick).strip('[]')
        pickList = pickList.strip("'")
        pickList = pickList.split(',')
        
        relevant_picks.append(pickList)

    return relevant_picks

# Returns the number of picks with an occurrence of an article
def findArticleOccurrences(picks, article):
    occurrences = 0

    for pick in picks:
        if article in pick:
            occurrences += 1

    return occurrences

def findCorrelation(df, article):
    XPicks = findPicksWithArticle(df, article)

    unique_articles = set()

    for order in XPicks:
        for item in order:
            unique_articles.add(item)

    unique_articles.remove(article)

    # Create an empty dictionary to store the correlation values for each article pair
    correlation_dict = {}

    for item in unique_articles:
        # Find all occurrences of item in the set of all picks containing the article
        subOccurrences = findArticleOccurrences(XPicks, item)
        # Express as a probability
        correlation = round((subOccurrences / len(XPicks)), 4) * subOccurrences
        # Add the calculated correlation value to the dictionary with the article pair as the key
        correlation_dict[(article, item)] = correlation

    return correlation_dict

with open('data/unique_articles.csv') as file_obj:
    correlation_obj = {}
    # Create reader object 
    reader_obj = csv.reader(file_obj)

    # Iterate over each row in the csv
    next(file_obj)
    for row in reader_obj:
        article_name = row[0]
        correlation_values = findCorrelation(df, article_name)
        XPicksLength = len(findPicksWithArticle(df, article_name))

        # Multiply the correlation value by XPicksLength for each row in the dictionary
        for article_pair, correlation in correlation_values.items():
            correlation_values[article_pair] = correlation * XPicksLength

        if correlation_values:
            correlation_obj.update(correlation_values)

# Create a correlation matrix and add new column
corr_matrix = pd.DataFrame.from_dict(correlation_obj, orient='index')
corr_matrix.columns = ['Correlation']

# Create separate columns for articles
corr_matrix[['Article 1', 'Article 2']] = pd.DataFrame(corr_matrix.index.tolist(), index=corr_matrix.index)

# Write correlation matrix to CSV file
corr_matrix.to_csv('data/correlation_matrix.csv', index=False)


In [34]:
# Sort the correlation matrix in a descending fashion based on correlation
df = pd.read_csv("data/correlation_matrix.csv")

sorted_df = df.sort_values(by=["Correlation"], ascending=False)
sorted_df.to_csv('data/correlation_matrix.csv', index=False)

In [35]:
# List all of the most popular items

# Read the pickHistoryOrderNo.csv file as a DataFrame
df = pd.read_csv('data/output_file.csv', dtype={"ARTNO": str})

# Create an empty dictionary to store the article pick count
article_count_dict = {}

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the ARTNO of the article picked in the current row
    article = row['ARTNO']
    # If the article is not in the dictionary, add it with a count of 1
    if article not in article_count_dict:
        article_count_dict[article] = 1
    # If the article is already in the dictionary, increment its count by 1
    else:
        article_count_dict[article] += 1

# Sort the dictionary by value in descending order and convert it to a list of tuples
sorted_article_count = sorted(article_count_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted article pick count in key:value format
for article, count in sorted_article_count:
    print(f"{article}: {count}")

df_popularity = pd.read_csv('data/popularity.csv')

# Sort the DataFrame in descending order based on the 'popularity' column
df_popularity = df_popularity.sort_values(by=['popularity'], ascending=False)

# Write the sorted DataFrame back to the 'popularity.csv' file
df_popularity.to_csv('data/popularity.csv', index=False)



30256891: 26
70277957: 21
00214505: 19
30263245: 18
30277959: 18
20214566: 16
70214559: 12
90256893: 12
40256895: 11
10246308: 10
90246352: 9
10257306: 8
00257298: 8
80275887: 7
90277961: 7
30214504: 7
80331930: 6
00428549: 6
10443974: 6
60442383: 6
80214568: 6
20207410: 6
20331198: 5
00473546: 5
70246386: 5
70346932: 5
70277981: 5
50214560: 5
70214564: 5
80261258: 4
00324518: 4
40497828: 4
70246683: 4
20275814: 4
50357002: 4
10357004: 4
20257315: 4
80256940: 4
20201724: 4
40277992: 4
70257638: 4
10331194: 3
60505511: 3
70330945: 3
70256931: 3
70238541: 3
80429757: 3
20278167: 3
80089239: 3
90347351: 3
20317079: 3
10510299: 3
70246348: 3
80214549: 3
40205599: 3
80524882: 3
50342237: 3
60396121: 3
90511105: 3
60509142: 3
40211977: 3
70277995: 3
10446883: 3
10368489: 3
90124534: 3
40295528: 3
60293566: 3
00461379: 3
20476346: 3
50277977: 3
10277984: 3
40201723: 3
30246722: 3
60263244: 3
60246396: 3
30404028: 3
50404032: 3
20246360: 3
20246708: 3
20393662: 3
50278726: 3
00257302: 3
102447

In [36]:
import pandas as pd

# Read output_file.csv as a DataFrame
df = pd.read_csv('data/output_file.csv', dtype={"ARTNO": str})

# Create an empty dictionary to store the article pick count
article_count_dict = {}

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the ARTNO of the article picked in the current row
    article = row['ARTNO']
    # If the article is not in the dictionary, add it with a count of 1
    if article not in article_count_dict:
        article_count_dict[article] = 1
    # If the article is already in the dictionary, increment its count by 1
    else:
        article_count_dict[article] += 1

# Read the unique_articles.csv file as a DataFrame
df_articles = pd.read_csv('data/unique_articles.csv', dtype={"articles": str}, on_bad_lines="skip")

# Create empty lists to store the search keys and corresponding popularity values
article_list = []
popularity_list = []

# Iterate over each row in the DataFrame
for index, row in df_articles.iterrows():
    # Get the article name from the 'articles' column
    article = row['articles']
    # If the article name is in the article count dictionary, append it to the search keys list and its popularity count to the popularity values list
    if article in article_count_dict:
        article_list.append(article)
        popularity_list.append(article_count_dict[article])

# Create a DataFrame with the search keys and corresponding popularity values as separate columns
df_popularity = pd.DataFrame({'ARTNO': article_list, 'popularity': popularity_list})

# Write the DataFrame to a new 'popularity.csv' file
df_popularity.to_csv('data/popularity.csv', index=False)


In [37]:
import pandas as pd

# load the pickHistoryOrderNo.csv file as a dataframe with the correct encoding
df = pd.read_csv('data/leanPicks.csv', encoding='iso-8859-1', on_bad_lines="skip")

# group the dataframe by 'ARTNO'
artno_groups = df.groupby('ARTNO')

# initialize an empty list to store the results
results = []

# iterate over each group and calculate the probabilities
for artno, group_df in artno_groups:
    num_truck = group_df[group_df['ORDER_TYPE'] == 'Truck'].shape[0]
    num_collect = group_df[group_df['ORDER_TYPE'] == 'Collect'].shape[0]
    total_rows = group_df.shape[0]
    
    # calculate the probabilities for each group
    prob_truck = num_truck / total_rows
    prob_collect = num_collect / total_rows
    prob_other = 1 - prob_truck - prob_collect
    
    # store the results for each group as a dictionary
    results.append({'ARTNO': artno, 'TRUCK': prob_truck, 'COLLECT': prob_collect, 'OTHER': prob_other})

# convert the list of dictionaries to a dataframe
result_df = pd.DataFrame(results)

# write the dataframe to a new CSV file
result_df.to_csv('data/probability_results.csv', index=False)


In [38]:
import pandas as pd

# load the probability_results.csv and popularity.csv files as dataframes
prob_df = pd.read_csv('data/probability_results.csv')
pop_df = pd.read_csv('data/popularity.csv')

# merge the two dataframes on the ARTNO column
merged_df = pd.merge(prob_df, pop_df, on='ARTNO')

# select the desired columns and rename them
new_df = merged_df[['ARTNO', 'popularity', 'TRUCK', 'COLLECT']].rename(columns={'TRUCK': 'Truck', 'COLLECT': 'Collect'})

# write the dataframe to a new CSV file
new_df.to_csv('data/article_data.csv', index=False)


In [39]:
import pandas as pd

# load the article_data.csv and correlation_matrix.csv files as dataframes
data_df = pd.read_csv('data/article_data.csv')
corr_df = pd.read_csv('data/correlation_matrix.csv')

# initialize an empty list to store the result dictionaries
result = []

# iterate through each row in data_df and create result dictionary
for index, row in data_df.iterrows():
    artno = row['ARTNO']
    popularity = row['popularity']
    truck = row['Truck']
    collect = row['Collect']
    correlations = {}
    
    # find all the correlation values associated with the artno
    corr_values1 = corr_df[corr_df['Article 1']==artno]
    corr_values2 = corr_df[corr_df['Article 2']==artno]
    corr_values = pd.concat([corr_values1, corr_values2])
    
    # create a dictionary of ARTNO and corresponding correlation value
    for index_corr, row_corr in corr_values.iterrows():
        if row_corr['Article 1'] == artno:
            correlations[str(row_corr['Article 2'])] = row_corr['Correlation']
        else:
            correlations[str(row_corr['Article 1'])] = row_corr['Correlation']
    
    # append the result dictionary to the results list
    result.append({'ARTNO': artno, 'popularity': popularity, 'Truck': truck, 'Collect': collect, 'Correlations': correlations})

# write the result list to a new CSV file
result_df = pd.DataFrame(result)
result_df.to_csv('data/article_data_with_correlations.csv', index=False)